# The Gender Wage Gap and Part-time workers

In this project, we will analyze the gender wage gap in various sectors in Denmark and how it has developed over time. Many other studies have shown that education and work experience do not have an impact on the gender wage gap. Therefore, we will also examine the share of female part-time workers in these sectors and how it has changed over the years. Women may be more likely to work part-time than men, often due to caregiving responsibilities or other factors.

We will use data from Statistics Denmark to conduct our analysis and visualize the results. By doing so, we hope to gain insights into the trends and patterns of the gender wage gap and part-time work in different sectors and understand the factors that may contribute to these differences.

First we need to modules and get acces to Statistics Denmark´s API. 

Imports and set magics:

In [1]:
# The DST API wrapper
%pip install git+https://github.com/elben10/pydst

  Cloning https://github.com/elben10/pydst to /private/var/folders/__/y1zwfmg97w56l4ffjlqcx5h00000gn/T/pip-req-build-409c94fa
  Running command git clone --filter=blob:none --quiet https://github.com/elben10/pydst /private/var/folders/__/y1zwfmg97w56l4ffjlqcx5h00000gn/T/pip-req-build-409c94fa
  Resolved https://github.com/elben10/pydst to commit bb347560a1b3c17684fda072c829658da38f5a72
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from ipywidgets import interact
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from tabulate import tabulate

# Import pydsta and setup data loader with the langauge 'english'
import pydst
dst = pydst.Dst(lang='en')

# 1. Read and Clean Data

To this project we uses data for Denmark from the Statistics Denmark's API. We uses data on wages and employment. 

**1.1 Hourly Wage per Sector**

We retrives data from the statistic 'LONS30' and stores the variables in the wage_var variable. This statistic contains data on earnings by sex, sector, and region. As we want the data in wide format, first we handle the data about the female wage, then the male wage. 

We store the data on female hourly wages by sector for all possible years in wage_k. We selects the variables to download with the variables argument, which is a dictionary where the keys are the variable names and the values are lists of the values to select. By using {'KØN': ['K']} we only select observatinos where the gender is female. We filter the data such we look at all salary earners in all of Denmark. We also choose to look at earnings in DKK per hour such part time worker do not  

As the focus for this project is limited, we do not use all the variables. So we drop some of them: 'OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL' and 'KØN'. By using 'axis=1' we drop columns, rather than rows. As we remove some variables the rows from these variables are still there, so we use groupby to aggregate the rows by year and sector. 
Then we rename the variabel to get more meaningful names, such as 'TID' is chaged to 'Year'. 

Exactly the same is done for the male hourly wage to have a short format instead of long.  

Finally, we merges the two DataFrames (wage_k and wage_m) by the "Year" and "Sector" columns, so that the resulting DataFrame wage contains columns for both female and male hourly wages. The how='left' argument means to keep all rows from wage_k (which contains the female wages) and only add matching rows from wage_m (which contains the male wages). The resulting DataFrame has columns for "Year", "Sector", "Female Hourly wage", and "Male Hourly wage".

In [3]:
# Female wage

# Get variables for the LONS30 table
wage_vars = dst.get_variables(table_id='LONS30')
wage_vars

# Download data
wage_k = dst.get_data(table_id = 'LONS30', 
                    variables={ 'Tid':['*'],
                                'KØN': ['K'], 
                                'OMRÅDE': ['000'],
                                'SEKTOR':['*'],
                                'AFLOEN':['TIFA'],
                                'LONGRP':['LTOT'],
                                'LØNMÅL':['FORINKL']})

# Drop columns
drop = ['OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL', 'KØN']
wage_k.drop(drop, axis=1, inplace=True)

# Rename columns
wage_k.rename(columns = {'TID':'Year', 
                      'SEKTOR': 'Sector',
                      'INDHOLD': 'Female Hourly wage'}, inplace=True)

# Show first few rows
wage_k.head()


,Year,Sector,Female Hourly wage
0,2013,All sectors,270.72
1,2013,Government including social security funds,301.64
2,2013,Municipal and regional government total,263.97
3,2013,Regional government,286.55
4,2013,Municipal government,256.89


In [4]:
# Male wage

# Download data
wage_m = dst.get_data(table_id = 'LONS30', 
                    variables={ 'Tid':['*'],
                                'KØN': ['M'], 
                                'OMRÅDE': ['000'],
                                'SEKTOR':['*'],
                                'AFLOEN':['TIFA'],
                                'LONGRP':['LTOT'],
                                'LØNMÅL':['FORINKL']})

# Drop columns
drop = ['OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL', 'KØN']
wage_m.drop(drop, axis=1, inplace=True)

# Rename columns
wage_m.rename(columns = {'TID':'Year', 
                      'SEKTOR': 'Sector',
                      'INDHOLD': 'Male Hourly wage'}, inplace=True)

# Show first few rows
wage_m.head()


,Year,Sector,Male Hourly wage
0,2013,All sectors,309.21
1,2013,Government including social security funds,315.15
2,2013,Municipal and regional government total,292.10
3,2013,Regional government,366.47
4,2013,Municipal government,272.08


In [5]:
# Merge the wage_k and wage_m data frames based on the 'Year' and 'Sector' columns using a left join. 
wage=pd.merge(wage_k,wage_m,how='left',on=['Year', 'Sector'])

# Display the first few rows of the merged data frame.
wage.head()

,Year,Sector,Female Hourly wage,Male Hourly wage
0,2013,All sectors,270.72,309.21
1,2013,Government including social security funds,301.64,315.15
2,2013,Municipal and regional government total,263.97,292.10
3,2013,Regional government,286.55,366.47
4,2013,Municipal government,256.89,272.08


**1.2 Extent of Working Time**

This code is downloading and cleaning data related to employment in Denmark, specifically focusing on full-time and part-time work. From Statistics Denmark we use data from 'RAS307'. 

It specifies the variables to include in the data and drops unnecessary columns (ARBOMFANG, KØN, and BRANCHE07). Then, it renames the remaining columns (TID, SEKTOR, and INDHOLD) to Year, Sector, and Female Full-time, respectively, before sorting the data by year and displaying the data. 

Next, we want to get data on part-time worker so with the variable ARBOMFANG set to 'D' to specify part-time work we repeate the code from full time workers. The code then drops unnecessary columns and renames the remaining columns to Year, Sector, and Female Part-time, respectively. 

The same is done for the male workers. 

In the end we merge the dataframes are merged on Year and Sector to create the time dataframe that contains the number of female and male employees, both full-time and part-time, in each sector for each year.


In [6]:
# Female Full-time

# Download data
time_k_ft = dst.get_data(table_id = 'RAS307', 
                    variables={'TID':['*'], 
                               'KØN':['K'], 
                               'BRANCHE07': ['*'],
                               'SEKTOR':['*'],
                               'ARBOMFANG':['H']})

# Drop columns
drop = ['ARBOMFANG', 'KØN','BRANCHE07']
time_k_ft.drop(drop, axis=1, inplace=True)

# Rename columns
time_k_ft.rename(columns = {'TID':'Year', 
                            'KØN':'Gender',
                            'BRANCHE07':'Industry', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Female Full-time'}, inplace=True)

# Show first few rows
time_k_ft = time_k_ft.sort_values(by='Year', ascending=True)
time_k_ft.head(10)

## Female Part-time

# Download data
time_k_pt = dst.get_data(table_id = 'RAS307', 
                    variables={'TID':['*'], 
                               'KØN':['K'], 
                               'BRANCHE07': ['*'],
                               'SEKTOR':['*'],
                               'ARBOMFANG':['D']})

# Drop columns
drop = ['ARBOMFANG', 'KØN','BRANCHE07']
time_k_pt.drop(drop, axis=1, inplace=True)

# Rename columns
time_k_pt.rename(columns = {'TID':'Year', 
                            'KØN':'Gender',
                            'ALDER': 'Age',
                            'BRANCHE07':'Industry', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Female Part-time'}, inplace=True)

# Show first few rows
time_k_pt = time_k_pt.sort_values(by='Year', ascending=True)
time_k_pt.head()


,Year,Sector,Female Part-time
385,2008,Regional government,2802
405,2008,Sector not stated,2
404,2008,Private corporations,9534
403,2008,Public corporations,53
402,2008,Municipal government,18


In [7]:
## Male Full-time

# Download data
time_m_ft = dst.get_data(table_id = 'RAS307', 
                    variables={'TID':['*'], 
                               'KØN':['M'], 
                               'BRANCHE07': ['*'],
                               'SEKTOR':['*'],
                               'ARBOMFANG':['H']})

# Drop columns
drop = ['ARBOMFANG', 'KØN','BRANCHE07']
time_m_ft.drop(drop, axis=1, inplace=True)

# Rename columns
time_m_ft.rename(columns = {'TID':'Year', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Male Full-time'}, inplace=True)

# Show first few rows
time_m_ft = time_m_ft.sort_values(by='Year', ascending=True)
time_m_ft.head(10)

## Male Part-time

# Download data
time_m_pt = dst.get_data(table_id = 'RAS307', 
                    variables={'TID':['*'], 
                               'KØN':['M'], 
                               'BRANCHE07': ['*'],
                               'SEKTOR':['*'],
                               'ARBOMFANG':['D']})

# Drop columns
drop = ['ARBOMFANG', 'KØN','BRANCHE07']
time_m_pt.drop(drop, axis=1, inplace=True)

# Rename columns
time_m_pt.rename(columns = {'TID':'Year', 
                            'KØN':'Gender',
                            'ALDER': 'Age',
                            'BRANCHE07':'Industry', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Male Part-time'}, inplace=True)

# Show first few rows
time_m_pt = time_m_pt.sort_values(by='Year', ascending=True)
time_m_pt.head()


,Year,Sector,Male Part-time
376,2008,Municipal government,261
397,2008,Public corporations,36
396,2008,Municipal government,5
395,2008,Central government,128
394,2008,Sector not stated,7


In [8]:
# Merge the time_k_ft and time_k_pt data frames based on the 'Year' and 'Sector' columns using a left join.
time_k=pd.merge(time_k_ft,time_k_pt,how='left',on=['Year', 'Sector'])

# Group the time_k data frame by the 'Year' and 'Sector' columns, and sum the values for each group.
time_k = time_k.groupby(['Year', 'Sector']).sum()

# Merge the time_m_ft and time_m_pt data frames based on the 'Year' and 'Sector' columns using a left join.
time_m=pd.merge(time_m_ft,time_m_pt,how='left',on=['Year', 'Sector'])

# Group the time_m data frame by the 'Year' and 'Sector' columns, and sum the values for each group.
time_m = time_m.groupby(['Year', 'Sector']).sum()

# Merge the time_k and time_m data frames based on the 'Year' and 'Sector' columns using a left join.
time=pd.merge(time_k,time_m,how='left',on=['Year', 'Sector'])

# 2. Exploring Data

**2.1 Hourly Wage by Gender and Sector**

We use the data that we downloaded and cleaned in section 1.1. We want to study how the wages have developeed over the years so we use the following code. 

The following code creates an interactive plot that shows the hourly wage of males and females for different sectors. Using the dropdown function, you can select the sector to examine.

In [9]:
# Group the data by year, sector, and gender, and calculate the mean hourly wage
wage_by_year_sector = wage.groupby(['Year', 'Sector']).mean().reset_index()

# Create the plot layout
layout = go.Layout(xaxis=dict(title='Year'),
                   yaxis=dict(title='Hourly wage (DKK)'))

# Define the dropdown options
sector_options = list(wage_by_year_sector['Sector'].unique())

# Define the callback function
def update_plot(sector):
    # Filter the data based on the selected sector
    filtered_data = wage_by_year_sector[wage_by_year_sector['Sector'] == sector]

    # Create traces for the full-time and part-time data
    trace_k = go.Scatter(x=filtered_data['Year'], y=filtered_data['Female Hourly wage'], mode='lines',line = dict(color='red', width=2), name='Female')
    trace_m = go.Scatter(x=filtered_data['Year'], y=filtered_data['Male Hourly wage'], mode='lines',line = dict(color='blue', width=2), name='Male')

    # Create the plot figure and add the traces and layout
    fig = go.Figure(data=[trace_k, trace_m], layout=layout)
    fig.update_layout(title='Hourly wage by gender and sector ({})'.format(sector))
    fig.show()

# Create the interactive dropdown
interact(update_plot, sector=sector_options)

interactive(children=(Dropdown(description='sector', options=('All sectors', 'Corporations and organizations',…

<function __main__.update_plot(sector)>

The graph shows a proportional increase in hourly wages across all sectors, with some differences in the rate of increase and the starting and ending wage levels. In particular, the female hourly wage started at 270.72 DKK and ended at 316.83 DKK in 2021, while the male hourly wage increased from 309.21 DKK in 2013 to 352.15 DKK in 2021. 

There are variations in hourly wages across different sectors. For example, we can see that the 'Municipal government' sector has a relatively narrow range of hourly wages. On the other hand, in the 'Regional government' sector, there is a much greater disparity in hourly wages, with a wider range of starting and ending wage levels.

**2.2 Extent of Working Time by Gender and Sector**

In this section we look at the gender differences in working time. Full-time employment is defined as a weekly working hours above 32 hours and part-time emplotment is defined as a weekly working hours equal or below 32 hours.

The code below calculates the average number of employees for each year and sector. We create a plot the shows the number female and male employees, each with full-time and part-time data. We also create a dropdown menu to choose a sector. The resulting plot shows the employment by gender and the chosen sector over the years, with separate lines for full-time (hard line) and part-time employment (dashdot line) for each gender.


In [10]:
# Group the data by year, sector, and industry and calculate the mean for each group
time_k_avg = time_k.groupby(['Year', 'Sector']).sum().reset_index()
time_m_avg = time_m.groupby(['Year', 'Sector']).sum().reset_index()

# Create the plot layout
layout2 = go.Layout(xaxis=dict(title='Year'),
                   yaxis=dict(title='Number of employees'))

# Define the dropdown options
sector_options_time = list(time_k_avg['Sector'].unique())

# Create the figure
fig = go.Figure(layout=layout2)

# Define the callback function
def update_plot(sector):
    # Filter the data based on the selected sector
    filtered_data_k = time_k_avg[time_k_avg['Sector'] == sector]
    filtered_data_m = time_m_avg[time_m_avg['Sector'] == sector]


    # Create traces for the full-time and part-time data
    trace_k_ft = go.Scatter(mode='lines', x=filtered_data_k['Year'], y=filtered_data_k['Female Full-time'],  line = dict(color='red', width=2), name='Female Full-time')
    trace_k_pt = go.Scatter(mode='lines', x=filtered_data_k['Year'], y=filtered_data_k['Female Part-time'], line = dict(color='red', dash='dashdot'), name='Female Part-time')

    trace_m_ft = go.Scatter(mode='lines', x=filtered_data_m['Year'], y=filtered_data_m['Male Full-time'], line = dict(color='blue', width=2), name='Male Full-time')
    trace_m_pt = go.Scatter(mode='lines', x=filtered_data_m['Year'], y=filtered_data_m['Male Part-time'], line = dict(color='blue', dash='dashdot'), name='Male Part-time')


    # Create the plot figure and add the traces and layout
    fig = go.Figure(data=[trace_k_ft, trace_k_pt, trace_m_ft, trace_m_pt], layout=layout2)
    fig.update_layout(title='Work Time by gender and sector ({})'.format(sector))
    fig.show()

# Create the interactive dropdown
interact(update_plot, sector=sector_options_time)

interactive(children=(Dropdown(description='sector', options=('Central government', 'Municipal government', 'P…

<function __main__.update_plot(sector)>

By looking at the numbers, for some sector the numbers are quite large. I think something went wrong when grouping the data and it then does not make sense to comment the graph. As we look later at the share of employees, it might not be a problem. I will look into that later. 

# 3. Merge data sets

The following code is used to merge the two separate datasets containing information on wages and employees. We uses the inner merging function which combines the datasets based on the intersection of matching values of specified columns. It keeps only the rows from each dataset that have matching values in the specified columns, and removes the rows that do not have matching values.

In [11]:
# Merge the dataframes
merged = pd.merge(wage,time, how='inner', on=['Year', 'Sector'])

# Show first few rows of merged dataframe
merged.head()

,Year,Sector,Female Hourly wage,Male Hourly wage,Female Full-time,Female Part-time,Male Full-time,Male Part-time
0,2013,Regional government,286.55,366.47,1077778,316176,267336,50596
1,2013,Municipal government,256.89,272.08,3341143,2072011,1377920,592382
2,2014,Regional government,290.88,369.17,1091376,309554,274584,50271
3,2014,Municipal government,262.80,278.22,4566618,2312585,1432471,596524
4,2015,Regional government,296.95,373.62,1171646,297598,274320,50232


# 4.    Analysis

**4.1 Descriptive Statistics**

First we look at some descriptive statistics for the wages to examine the differences between the gender.

In [12]:
# Define the function for generating the table
def describe_stats(year):
    
    # Filter the employment data by year and gender
    merged_filtered = merged[(merged['Year'] == year)]
    
    # Calculate the employment rate statistics
    merged_stats = merged_filtered['Female Hourly wage'].describe()
    merged_stats2 = merged_filtered['Male Hourly wage'].describe()

    # Format the statistics as a list of tuples
    table_data = [('Count', f'{merged_stats["count"]:.2f}'),
                  ('Mean', f'{merged_stats["mean"]:.2f}'),
                  ('Std Dev', f'{merged_stats["std"]:.2f}'),
                  ('Min', f'{merged_stats["min"]:.2f}'),
                  ('25%', f'{merged_stats["25%"]:.2f}'),
                  ('50%', f'{merged_stats["50%"]:.2f}'),
                  ('75%', f'{merged_stats["75%"]:.2f}'),
                  ('Max', f'{merged_stats["max"]:.2f}')]
    
    # Format the statistics as a list of tuples
    table_data2 = [('Count', f'{merged_stats2["count"]:.2f}'),
                  ('Mean', f'{merged_stats2["mean"]:.2f}'),
                  ('Std Dev', f'{merged_stats2["std"]:.2f}'),
                  ('Min', f'{merged_stats2["min"]:.2f}'),
                  ('25%', f'{merged_stats2["25%"]:.2f}'),
                  ('50%', f'{merged_stats2["50%"]:.2f}'),
                  ('75%', f'{merged_stats2["75%"]:.2f}'),
                  ('Max', f'{merged_stats2["max"]:.2f}')]
    
    # Print the table using the tabulate library
    print(tabulate(table_data,  headers=['', f'Female Hourly wage in {year}'], 
                                    tablefmt='fancy_grid'))
    print(tabulate(table_data2, headers=['', f'Male Hourly wage in {year}'], 
                                    tablefmt='fancy_grid'))
   
# Define the interact function
interact(describe_stats, year=IntSlider(min=merged['Year'].min(), max=merged['Year'].max(), step=1, value=merged['Year'].min()))

interactive(children=(IntSlider(value=2013, description='year', max=2021, min=2013), Output()), _dom_classes=(…

<function __main__.describe_stats(year)>

By the tables above, we see summary statistics of the female and male hourly wages. By using the slider, we find that in 2013 the average hourly wage for the two sectors was 271.72 DKK for women and 319.27 DKK for men. It means that the women on average earn 85,11 pct. of men´s earnings. In 2021 the female hourly was has increased to 321.05 DKK and the male hourly wage is 355.13 DKK. The female hourly wage has increased more than the male wage such that for the two sectors women on average earn 90.40 pct. of men´s earnings in 2021. 

**4.2 The Gender Wage Gap**

To see if one of the sectors are driving the reducing in the gender wage differences, we plot the gender wage gap over the years.

First, the code below is calculating the gender wage gap as a percentage by dividing the hourly wage of female workers by the hourly wage of male workers. 
Then we create a a line plot to show the gender wage gap over time for each sector, where the x-axis represents the years and the y-axis represents the gender wage gap.

In [13]:
# calculating the gender wage gap
merged['Gender wage gap'] = 100 - (merged['Female Hourly wage'] / (merged['Male Hourly wage'])) * 100

# create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged, x='Year', y='Gender wage gap', color='Sector', title='Gender Wage Gap by Sector Over Time')

# add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Gender Wage Gap (%)')

# display the plot
fig.show()

The plot above shows the gender wage gap in the municipal and regional government sectors. The data indicates that the wage gap between men and women is significantly smaller in the municipal government sector compared to the regional government sector. Specifically, the graph shows the gender wage gap in the regional government sector was much higher, at 21.81 in 2013, but has been decreasing over time. By 2021, the wage gap had decreased by almost 40% to 13.64. That the gender wage gap equals 13.64 pct. means that the female workers on average earns 13.64 pct. less than men in the same sector. 

On the other hand, the gender wage gap in the municipal government sector was 5.58 in 2013 and remained relatively stable throughout the period. In 2021, the gender wage gap in the municipal government sector was 4.54, which is much lower than the national average wage gap of around 15%.

According to the graph presented in section 2.1, the wage of female employees in the regional government sector has increased more than male wage. This finding supports the observation that the gender wage gap in this sector has decreased over time.

**4.3 Female Work and Working Time**

Next, we look at the share of female workers in the two sectors. First we calculate the share of female workers as a percentage by the sum of female full-time and part-time workers and dividing with the total number of female workers. Then we create use Plotly library to create a line plot that shows the share of female workers over time for each sector, where the x-axis represents the years and the y-axis represents the percentage of female workers. The plot is also color-coded based on the different sectors.

In [14]:
# calculating the share of female workers
merged['Share of Female Workers'] = (merged['Female Full-time'] + merged['Female Part-time']) / (merged['Male Full-time'] + merged['Male Part-time'] + merged['Female Full-time'] + merged['Female Part-time']) * 100

# create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged,   x='Year', 
                        y='Share of Female Workers', 
                        color='Sector', 
                        title='Share of Female workers')

# add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Share of Female Workers (%)')

# display the plot
fig.show()


The graph illustrates the percentage of female workers in the regional and municipal government sectors over time. Throughout the entire time period, the share of female workers was higher in the regional government sector than in the municipal government sector. However, the share has slightly decreased over the years, reaching a minimum of 79.08 pct. in 2017. On the other hand, the female share for the municipal government sector has varied significantly, with a maximum of 77.96 pct. observed in 2015. 

To get a better understanding of the female employment, we examine the female share of part-time workers which is calculated as female part-time workers divided by the total of female workers. We show the share in a line plot. 

In [15]:
# calculating the share of female part-time workers
merged['Share of Female Part-time Workers'] = (merged['Female Part-time']) / (merged['Female Full-time'] + merged['Female Part-time']) * 100

# create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged,   x='Year', 
                        y='Share of Female Part-time Workers', 
                        color='Sector', 
                        title='Share of Female Part-time Workers')

# add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Share of Female Part-time Workers (%)')

# display the plot
fig.show()

The graph shows the share of female part-time workers in the Municipal and Regional Government sectors. It is highest in the Municipal Government sector, decreasing from 38.27 pct. to 31.78 pct. between the years, indicating a declining trend. On the other hand, the share of female part-time workers in the Regional Government sector remains relatively stable, varying between 19.43 pct. and 22.68 pct. during the same period.

The data indicates that, although the regional government sector has the largest overall share of female workers, it has the smallest share of female part-time workers. On the other hand, the municipal government sector has a relatively smaller share of female workers but a significantly higher proportion, around 35%, work part-time.

This suggests that the two sectors have different labor market dynamics, with the municipal government sector offering more part-time work opportunities for female employees compared to the regional government sector. It could be inferred that the regional government sector may have a preference for full-time workers, while the municipal government sector may offer more flexibility in terms of working hours.


# 5. Conclusion

In conclusion, this project analyzed the gender wage gap and the share of female workers in different sectors in Denmark. Using data from Statistics Denmark the results showed that the gender wage gap has decreased over time in both sectors, but it was significantly smaller in the municipal government sector. Additionally, the share of female workers was higher in the regional government sector, but the municipal government sector had a higher proportion of female part-time workers. These findings suggest that the two sectors have different labor market dynamics, with the municipal government sector offering more part-time work opportunities for female employees compared to the regional government sector. 

Overall, this analysis provides insights into the trends and patterns of gender wage gap and part-time work in different sectors and can help us understand the factors that contribute to these differences.